In [4]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random 

In [5]:
initial_board = np.zeros(shape=(5,5), dtype=np.int32)
print(initial_board)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [8]:
class State:
    def __init__(self):
        #self.father = None
        self.board = np.zeros(shape=(5,5), dtype=np.int32)
        self.player= None
        
class Player:
    def __init__(self):
        self.value = 0
        #self.moves = 0
        self.name = None
        
        
def result(state):
    #Horizontales
    for i in range(5):
        if np.all(state.board[i][0:4] == 1) or np.all(state.board[i][1:5] == 1):
            return 1
        elif np.all(state.board[i][0:4] == -1) or np.all(state.board[i][1:5] == -1):
            return -1
        
    #Verticales
    for i in range(5):
        column = state.board[:,i]
        if np.sum(column == 1) == 4:
            if np.all(column[0:4] == 1) or np.all(column[1:5] == 1):
                return 1
        elif np.sum(column == -1) == 4:
            if np.all(column[0:4] == -1) or np.all(column[1:5] == -1):
                return -1
            
    #Diagonales       
    for i in range(-1,2):
            diag = state.board.diagonal(i)
            oppdiag = np.fliplr(state.board).diagonal(i)

            if np.sum(diag == 1) == 4 or np.sum(oppdiag == 1) == 4:
                if np.all(diag[0:4] == 1) or np.all(oppdiag[0:4] == 1):
                    return 1
                if len(diag)>4:
                    if np.all(diag[1:5] == 1) or np.all(oppdiag[1:5] == 1):
                        return 1
            elif np.sum(diag == -1) == 4 or np.sum(oppdiag == -1) == 4:
                if np.all(diag[0:4] == -1) or np.all(oppdiag[0:4] == -1):
                    return -1
                if len(diag)>4:
                    if np.all(diag[1:5] == -1) or np.all(oppdiag[1:5] == -1):
                        return -1
                    
    #Cuadrado               
    for i in range(4):
                for j in range(4):
                    c = [state.board[i][j:j+2],state.board[i+1][j:j+2]]
                    cube = np.array(c)

                    if np.all(cube == 1):
                        return 1
                    elif np.all(cube == -1):
                        return -1

    return 0
        
def question_place(player):
    print(f'{player.name} Ingresa la posicion xy:')
    pos = input()
    x_pos, y_pos = transform_values(pos)
    return x_pos, y_pos


def question_move_place(player):
    print(f'{player.name} Ingresa la posicion de la ficha a mover xy:')
    pos = input()
    x_pos, y_pos = transform_values(pos)
    print(f'{player.name} Ingresa la posicion a la cual se movera xy:')
    pos = input()
    new_x_pos, new_y_pos = transform_values(pos)
    return x_pos, y_pos, new_x_pos, new_y_pos

def validate_place(x_pos, y_pos, board):
    if(x_pos>=0 and x_pos<5 and y_pos>=0 and y_pos<5):
        if(board[x_pos][y_pos]==0):
            return True
    return False

#def validate_new_place(x_pos, y_pos, board):
    #if(x_pos>=0 and x_pos<5 and y_pos>=0 and y_pos<5):
       # if(board[x_pos][y_pos]==0):
           # return True
   # return False
        
def init_Game(player1,player2):
    start_state = State()
    board = start_state.board
    
    for i in range(4):
        #Player1
        x_pos,y_pos=question_place(player1)
        while(validate_place(x_pos,y_pos, board)==False):
            print("lugar invalido.. vuelve a intentar")
            x_pos,y_pos=question_place(player1)
        board[x_pos][y_pos]=player1.value
        print(board)
        
        #Player2
        x_pos,y_pos=question_place(player2)
        while(validate_place(x_pos,y_pos, board)==False):
            print("lugar invalido.. vuelve a intentar")
            x_pos,y_pos=question_place(player2)
        board[x_pos][y_pos]=player2.value
        print(board)
        
    return board
        
        
def move(x_pos, y_pos, new_x_pos, new_y_pos, player, state):
    if (x_pos==new_x_pos and y_pos==new_y_pos):
        print("Posiciones iguales, vuelva a intentar..")
        return False
    if (state.board[x_pos][y_pos]==player.value):
        if(state.board[new_x_pos][new_y_pos]==0):
            adjacentPlaces = possible_moves(x_pos, y_pos, state.board)
            if [new_x_pos, new_y_pos] in adjacentPlaces:
                state.board[new_x_pos][new_y_pos]=player.value
                state.board[x_pos][y_pos]=0
                return True
            else:
                print("Destino no adyacente, vuelva a intentar..")
                return False
        else:
            print("Destino ocupado, vuelva a intentar..")
            return False
    else:
        print("ficha seleccionada vacia o del jugador oponente, vuelva a intentar..")
        return False

def possible_moves(x_pos, y_pos, board):
    validMoves = []
    AllMoves = [[0, -1], [0, +1], [-1, 0] ,[+1, 0], [-1, +1], [+1, -1], [-1, -1], [+1, +1]]

    for i in AllMoves:
        x = x_pos + i[0]
        y = y_pos + i[1]

        if( 0 <= x <= 4 and 0 <= y <= 4):
            if(validate_place(x,y, board)==True):
                validMoves.append([x,y]) 
    return validMoves

        
def game(player1,player2,movelimits):
    state = State()
    state.board = init_Game(player1,player2)
    print("Fase 2 Mover Fichas")
    for i in range(movelimits):
        #player 1
        state.player=player1
        x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player1)
        while(move(x_pos, y_pos, new_x_pos, new_y_pos, player1, state)==False):
            x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player1)
        print(state.board)
        res=result(state)
        if(res!=0):
            print("gano: ",res)
            break
        
        #player 2
        state.player=player2
        x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player2)
        while(move(x_pos, y_pos, new_x_pos, new_y_pos, player2, state)==False):
            x_pos, y_pos, new_x_pos, new_y_pos=question_move_place(player2)
        print(state.board)
        res=result(state)
        if(res!=0):
            print("gano: ",res)
            break
    
def transform_values(pos):
    rows = {"A":0,"B":1,"C":2,"D":3,"E":4}
    columns = {"1":0,"2":1,"3":2,"4":3,"5":4}  
    x_pos=rows[pos[0]]
    y_pos=columns[pos[1]]
    return x_pos, y_pos

In [9]:
p1=Player()
p1.value=1
p1.name="Max"
p2=Player()
p2.value=-1
p2.name="Min"
movelimits=3

game(p1,p2,movelimits)

Max Ingresa la posicion xy:
A1
[[1 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
Min Ingresa la posicion xy:
C1
[[ 1  0  0  0  0]
 [ 0  0  0  0  0]
 [-1  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Max Ingresa la posicion xy:
A2
[[ 1  1  0  0  0]
 [ 0  0  0  0  0]
 [-1  0  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Min Ingresa la posicion xy:
C2
[[ 1  1  0  0  0]
 [ 0  0  0  0  0]
 [-1 -1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Max Ingresa la posicion xy:
B1
[[ 1  1  0  0  0]
 [ 1  0  0  0  0]
 [-1 -1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Min Ingresa la posicion xy:
C3
[[ 1  1  0  0  0]
 [ 1  0  0  0  0]
 [-1 -1 -1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Max Ingresa la posicion xy:
B3
[[ 1  1  0  0  0]
 [ 1  0  1  0  0]
 [-1 -1 -1  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0  0  0]]
Min Ingresa la posicion xy:
D3
[[ 1  1  0  0  0]
 [ 1  0  1  0  0]
 [-1 -1 -1  0  0]
 [ 0  0 -1  0  0]
 [ 0  0  0  0  0]]
Fase 2 Mover Fichas
Max Ingresa la posicion de la